In [1]:
import wallycore as w

In [2]:
PREFIXES = {
        "LIQUID" : {"P2PKH": 57, "P2SH" : 39, "BECH32" : "lq", "BLINDED": 12},
        "LIQUID-REGTEST" : {"P2PKH": 235, "P2SH" : 75, "BECH32" : "tb", "BLINDED": 4}, 
}

In [3]:
def find_address_type(address_string):
    for network in PREFIXES:
        if PREFIXES[network]["BECH32"] == address_string[:2].lower():
            print(address_string)
            return (network, "BECH32")

    try:
        address_bytes = w.base58_to_bytes(address_string, w.BASE58_FLAG_CHECKSUM)
        if len(address_bytes) != 21 and len(address_bytes) != 55:
            raise Exception("Invalid length")
        first_byte = address_bytes[0]
        for network in PREFIXES:
            for address_type in PREFIXES[network]:
                if PREFIXES[network][address_type] == first_byte:
                    return (network, address_type)
    except:
        pass
    
    raise Exception("Invalid address")

In [4]:
def extract_data_bech32(address_string, network):
    return w.addr_segwit_to_bytes(address_string, PREFIXES[network]["BECH32"], 0)

def extract_data(address_string): 
    address_bytes = w.base58_to_bytes(address_string, w.BASE58_FLAG_CHECKSUM)
    return address_bytes[1:]

def unpack_blinded(address_string, network):
    return (w.confidential_addr_to_addr(address_string, PREFIXES[network]["BLINDED"]), 
           w.confidential_addr_to_ec_public_key(address_string, PREFIXES[network]["BLINDED"]))

In [5]:
def get_script_pub_key(address_string):
    (network, address_type) = find_address_type(address_string)
    if address_type == "BECH32":
        #TODO: currently unsupported in wally
        raise Exception("Unsupported")
    
    pubkey = None
    if address_type == "BLINDED":
        (address_string, pubkey) = unpack_blinded(address_string, network)
        (network, address_type) = find_address_type(address_string)
        
    data = extract_data(address_string)
    if address_type == "P2PKH":
        #<ScriptPubKey=OP_DUP OP_HASH160<Public KeyHash> OP_EQUAL OP_CHECKSIG
        return (bytearray([w.OP_DUP, w.OP_HASH160, 20]) + data + bytearray([w.OP_EQUAL, w.OP_CHECKSIG]), pubkey, network) 
        
    if address_type == "P2SH":
        #<ScriptPubKey=OP_HASH160<ScriptHash> OP_EQUAL
        return (bytearray([w.OP_HASH160, 20]) + data + bytearray([w.OP_EQUAL]), pubkey, network)


In [6]:
testcases = ["QLFdUboUPJnUzvsXKu83hUtrQ1DuxyggRg", 
             "VJLK52qZR5tgHBAQzqtwSDq9qSUpy1CWQd2KbMGvaZA9jKmEcxRnEUXiMb9jijFSMbC3TKzbRbLLcDjo",
             "QKAQRoVNcXqZ9ZUQ5vv8qqTgZXfYWvJyDM",
             "Pz7eBTfLoVaisE57DLn6rZhEtFNa7SnZ2Z",
             "CTEnQXZL65tp6pDg27fQB8GYuZfpgkW798A6qCm3zDpbWu6jDbjxP2cAeN2GEiYJ41dgbBPXY5XUjU7T"]

for address in testcases:
    (script, pubkey, network) = get_script_pub_key(address)
    print(address, "->", w.hex_from_bytes(script), "UNBLINDED" if pubkey is None else w.hex_from_bytes(pubkey), network)

QLFdUboUPJnUzvsXKu83hUtrQ1DuxyggRg -> 76a914fc26751a5025129a2fd006c6fbfa598ddd67f7e187ac UNBLINDED LIQUID
VJLK52qZR5tgHBAQzqtwSDq9qSUpy1CWQd2KbMGvaZA9jKmEcxRnEUXiMb9jijFSMbC3TKzbRbLLcDjo -> a9146cf0e62c6767358928dd2f63f0640ddef21743d187 03f349509ef21c2f6fca81dfc04df7219b234b6529f503f0431e5a64a1acacd062 LIQUID
QKAQRoVNcXqZ9ZUQ5vv8qqTgZXfYWvJyDM -> 76a914f0315c15ef24284b4804f80becf14844048e6fe987ac UNBLINDED LIQUID
Pz7eBTfLoVaisE57DLn6rZhEtFNa7SnZ2Z -> 76a9141f4133bed29660d80240ab1a4cb5b09ddefd1b8787ac UNBLINDED LIQUID
CTEnQXZL65tp6pDg27fQB8GYuZfpgkW798A6qCm3zDpbWu6jDbjxP2cAeN2GEiYJ41dgbBPXY5XUjU7T -> 76a91449d76db89c4452e05787d1bb54b21e2643c7d24a87ac 0264816a2cba23943a0d8618062eb0c5e6e18ed3c3b3654ca9df46d440310c8948 LIQUID-REGTEST
